# Excercise 1

Find the dataset from this year with the highest number of frames.

- What is the dataset's Product ID?
- How many frames does the dataset have?
- Is the dataset embargoed? If yes, when does the embargo period end?

**Tips:**

- You can pass `a.Provider("dkist")` to `Fido.search()` to search only for data provided by DKIST
- You can use `.keys()` or `.colnames` on a `DKISTQueryResponse` object to see all the column names

In [ ]:
from sunpy.net import attrs as a, Fido
import dkist.net

In [ ]:
# Search for all DKIST data this year
res = Fido.search(a.Provider("dkist"), a.Time("2025-01-01", "2026-01-01"))
res

In [ ]:
# Just get the DKIST results table and sort by number of frames, largest first
res = res['dkist']
res.sort("Number of Frames", reverse=True)

In [ ]:
# Print top result, ie dataset with largest number of frames
res[0]["Product ID", "Instrument", "Embargoed", "Number of Frames", "Embargo End Date"]

# Exercise 2

Load the sample VISP dataset. **Without actually slicing the dataset**, figure out how many files are in the following slices:

- `ds[0]`
- `ds[:, :200]`
- `ds[:, :200, :200]`
- `ds[:, :, :200, :200]`

In [ ]:
from dkist import load_dataset
from dkist.data.sample import VISP_L1_KMUPT

ds = load_dataset(VISP_L1_KMUPT)

In [ ]:
ds.shape

In [ ]:
ds.files

In [ ]:
ds[0].files

In [ ]:
ds[:, :200].files

In [ ]:
ds[:, :200, :200].files

In [ ]:
ds[:, :, :200, :200].files

# Exercise 3

Open the VBI sample dataset again. Choose two adjacent tiles and calculate the overlap between them in arcseconds.

In [ ]:
from dkist.data.sample import VBI_L1_NZJTB
from dkist import load_dataset
import astropy.units as u

ds = load_dataset(VBI_L1_NZJTB)

In [ ]:
# We only need the first image of each tile
first_tiles = ds.slice_tiles[0]

In [ ]:
# The bottom left tile is 0,0, get the corner pixel on the right side
tile1_bottom_right_pix = first_tiles[0,0].wcs.array_index_to_world(0, 4096)
tile1_bottom_right_pix

In [ ]:
# The bottom middle tile is 1,0, get the corner pixel on the left side
tile2_bottom_left_pix = first_tiles[1,0].wcs.array_index_to_world(0, 0)
tile2_bottom_left_pix

In [ ]:
(tile1_bottom_right_pix.separation(tile2_bottom_left_pix).to(u.arcsec),

tile1_bottom_right_pix.Tx - tile2_bottom_left_pix.Tx,

tile1_bottom_right_pix.Ty - tile2_bottom_left_pix.Ty)

# Exercise 4

Using Fido, find the co-temporal VISP dataset, VBI dataset and AIA image from 19:47 on 2024/04/17. Use `sunpy.visualization.drawing.extent` to plot the FOV of the VBI and VISP datasets onto the AIA image.

In [ ]:
# Download one VISP and one VBI metadata ASDF file and a co-temporal AIA image for 19:47 on 2024-04-17
# Using sunpy.visualization.drawing.extent plot the FOV of the VBi and VISP datasets on the AIA image.
import matplotlib.pyplot as plt

import sunpy.map
from sunpy.net import Fido, attrs as a
from sunpy.visualization import drawing

import dkist
import dkist.net

In [ ]:
res = Fido.search(a.Time("2024-04-17T19:47:00", "2024-04-17T19:48:00"),
                  a.Instrument.vbi | a.Instrument.visp | a.Instrument.aia)
# Download the first result from each provider
files = Fido.fetch(res[:,0])
files.sort()

In [ ]:
# Unpack the filenames
vbi_file, visp_file, aia_file = files

# Load VBI and VISP datasets
vbi, visp = dkist.load_dataset((vbi_file, visp_file))

# Load the AIA data
aia = sunpy.map.Map(aia_file)

In [ ]:
# Create an axes with the AIA data's coordinate frame
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=aia)

# Plot the AIA data
aia.plot(axes=ax)

# Draw the extents of the VBI and VISP data
drawing.extent(ax, vbi[0].wcs, color="C1", label="VBI")
drawing.extent(ax, visp[0,:,0,:].wcs, color="C2", label="VISP")

plt.legend()

_ = ax.axis((1000, 2000, 2600, 3400))